# Retail Product Classification- Entity Resolution from Text

This solution demonstrates how data harmonization and entity resolution can be achieved using Snowflake's AI features.
We will be streamlining the process by quickly generating proposed product matches, and using LLM functions to validate them.

In [ ]:
# Import python packages
import streamlit as st

from snowflake.core import Root

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session

session = get_active_session()
root = Root(session)
session.query_tag = {"origin":"sf_sit-is",
                     "name":"entity_resolution",
                     "version":{"major":1, "minor":0},
                     "attributes":{"is_quickstart":0, "source":"notebook"}}

db = session.get_current_database().strip('"')
wh = session.get_current_warehouse().strip('"')
prefix = session.get_current_warehouse().strip('"').split("_DS_WH")[0]

In [ ]:
sql_command = f"ALTER WAREHOUSE {wh} SET WAREHOUSE_SIZE = 'XXLarge'"
session.sql(sql_command).collect()

In [ ]:
session.sql(f"USE ROLE ACCOUNTADMIN").collect()
session.sql(f"CREATE SCHEMA IF NOT EXISTS {db}.MATCH").collect()
session.sql(f"USE SCHEMA {db}.MATCH").collect()

### Preparing Datasets
To begin, we'll be creating tables from a product dataset from Similarweb Ltd, a provider on Snowflake Marketplace. Specifically, we'll be focused on products from Office Depot and Staples, which are more likely to have product matches for our use case. 

As we create these tables, we'll be cleaning the product titles then generating vector embeddings from them using one of Snowflake's `CORTEX.EMBED_TEXT_768` function!

In [ ]:
CREATE OR REPLACE TABLE staples_products AS
SELECT 
    SITE,
    PRODUCT,
    TITLE,
    REGEXP_REPLACE(TITLE, '[^a-zA-Z0-9 ]', '') AS CLEANED_TITLE,
    BRAND,
    MAIN_CATEGORY,
    REGEXP_REPLACE(SUB_CATEGORY, '^Home - ', '') AS SUB_CATEGORY,
    SUM(ESTIMATED_VIEWS) AS ESTIMATED_VIEWS,
    SUM(ESTIMATED_PURCHASES) AS ESTIMATED_PURCHASES,
    SNOWFLAKE.CORTEX.EMBED_TEXT_768('snowflake-arctic-embed-m', CLEANED_TITLE) AS cleaned_title_embedding
FROM AMAZON_AND_ECOMMERCE_WEBSITES_PRODUCT_VIEWS_AND_PURCHASES.datafeeds.product_views_and_purchases
WHERE SITE like 'staples.com'
    AND TITLE not like '%N/A%'
    AND TITLE not like ''
    AND BRAND not like '%N/A%'
    AND MAIN_CATEGORY not like '%N/A%'
    AND SUB_CATEGORY not like '%N/A%'
GROUP BY PRODUCT, TITLE, CLEANED_TITLE, SITE, BRAND, MAIN_CATEGORY, SUB_CATEGORY;

In [ ]:
SELECT *
FROM STAPLES_PRODUCTS
limit 50;

In [ ]:
CREATE OR REPLACE TABLE officedepot_products AS
SELECT 
    SITE,
    PRODUCT,
    TITLE,
    REGEXP_REPLACE(TITLE, '[^a-zA-Z0-9 ]', '') AS CLEANED_TITLE,
    BRAND,
    MAIN_CATEGORY,
    SUB_CATEGORY,
    SUM(ESTIMATED_VIEWS) AS ESTIMATED_VIEWS,
    SUM(ESTIMATED_PURCHASES) AS ESTIMATED_PURCHASES,
    SNOWFLAKE.CORTEX.EMBED_TEXT_768('snowflake-arctic-embed-m', CLEANED_TITLE) AS cleaned_title_embedding
FROM AMAZON_AND_ECOMMERCE_WEBSITES_PRODUCT_VIEWS_AND_PURCHASES.datafeeds.product_views_and_purchases
WHERE SITE like 'officedepot.com'
    AND TITLE not like '%N/A%'
    AND TITLE not like ''
    AND BRAND not like '%N/A%'
    AND MAIN_CATEGORY not like '%N/A%'
    AND SUB_CATEGORY not like '%N/A%'
GROUP BY PRODUCT, TITLE, CLEANED_TITLE, SITE, BRAND, MAIN_CATEGORY, SUB_CATEGORY;

In [ ]:
SELECT TITLE, CLEANED_TITLE
from OFFICEDEPOT_PRODUCTS
limit 5;

### Vector Cosine Similarity
Here, we are creating a table of the vector cosine similarity scores between the two product tables using their brands as a prerequisite. This narrows down the amount of computing time significantly, and doesn't lose us any matches in the process.

In [ ]:
CREATE OR REPLACE TABLE SIMILARITY_SCORES AS
SELECT
    s.PRODUCT AS STAPLES_PRODUCT_CODE,
    s.TITLE AS STAPLES_TITLE,
    s.CLEANED_TITLE AS STAPLES_CLEANED_TITLE,
    s.BRAND AS STAPLES_BRAND,
    s.MAIN_CATEGORY AS STAPLES_MAIN_CATEGORY,
    s.SUB_CATEGORY AS STAPLES_SUB_CATEGORY,
    s.ESTIMATED_VIEWS AS STAPLES_EST_VIEWS,
    s.ESTIMATED_PURCHASES AS STAPLES_EST_PURCHASES,
    od.PRODUCT AS OFFICE_PRODUCT_CODE,
    od.TITLE AS OFFICE_TITLE,
    od.CLEANED_TITLE AS OFFICE_CLEANED_TITLE,
    od.BRAND AS OFFICE_BRAND,
    od.MAIN_CATEGORY AS OFFICE_MAIN_CATEGORY,
    od.SUB_CATEGORY AS OFFICE_SUB_CATEGORY,
    od.ESTIMATED_VIEWS AS OFFICE_DEPOT_EST_VIEWS,
    od.ESTIMATED_PURCHASES AS OFFICE_DEPOT_EST_PURCHASES,
    VECTOR_COSINE_SIMILARITY(s.CLEANED_TITLE_EMBEDDING, od.CLEANED_TITLE_EMBEDDING) AS SIMILARITY_SCORE
FROM STAPLES_PRODUCTS s
INNER JOIN OFFICEDEPOT_PRODUCTS od
    ON s.BRAND = od.BRAND;

### Similarity Score Threshold - 0.9

Here we are setting a minimum score for the product pairs to further consider them as viable matches.

Note: Product pairs at lower thresholds such as 0.8 are still potential matches, but the runtime of this notebook rapidly increases to over an hour as the threshold is lowered. You are free to experiment with the thresholds, just keep an eye on the runtime!

In [ ]:
SELECT * 
FROM SIMILARITY_SCORES
WHERE SIMILARITY_SCORE > 0.9
ORDER BY SIMILARITY_SCORE DESC
limit 100;

Since we only want to match each product once, let's first take the product with the highest similarity score:

In [ ]:
CREATE OR REPLACE TABLE TOP_MATCHES AS
SELECT
    STAPLES_PRODUCT_CODE AS STAPLES_ID,
    STAPLES_TITLE,
    STAPLES_BRAND,
    STAPLES_MAIN_CATEGORY,
    STAPLES_SUB_CATEGORY,
    STAPLES_EST_VIEWS,
    STAPLES_EST_PURCHASES,
    OFFICE_PRODUCT_CODE AS OFFICE_ID,
    OFFICE_TITLE,
    OFFICE_BRAND,
    OFFICE_MAIN_CATEGORY,
    OFFICE_SUB_CATEGORY,
    OFFICE_DEPOT_EST_VIEWS,
    OFFICE_DEPOT_EST_PURCHASES,
    SIMILARITY_SCORE
FROM SIMILARITY_SCORES
QUALIFY ROW_NUMBER() OVER (PARTITION BY STAPLES_PRODUCT_CODE ORDER BY SIMILARITY_SCORE DESC) <= 1;

In [ ]:
CREATE OR REPLACE TABLE MATCHED_PRODUCTS AS
SELECT *
FROM TOP_MATCHES
WHERE SIMILARITY_SCORE > 0.9;

## AI-driven Entity Resolution

### Here we use Mistral via Snowflake's `SNOWFLAKE.CORTEX.COMPLETE` function to evaluate matches and provide structured outputs based on its decision.
The model is taking into consideration both product descriptions and brands to ensure they are truly an identical match. It returns a few different values for us:
- A boolean indicating if the pair is truly a match down to every detail
- A string containing either a unified product title or a list of differences between the product based on the boolean
- A unified brand for both products
- A unified sub-category


Note: A variety of models were experimented with when developing this solution, with mistral-large performing best. You may experiment with others and alter the prompt as needed!

In [ ]:
CREATE OR REPLACE TABLE REVIEWED_MATCHES AS
SELECT
    m.STAPLES_ID,
    m.STAPLES_TITLE,
    m.STAPLES_BRAND,
    REGEXP_REPLACE(m.STAPLES_SUB_CATEGORY, '^Home - ', '') AS STAPLES_SUB_CATEGORY,
    m.STAPLES_EST_VIEWS,
    m.STAPLES_EST_PURCHASES,
    m.OFFICE_ID,
    m.OFFICE_TITLE,
    m.OFFICE_BRAND,
    REGEXP_REPLACE(m.OFFICE_SUB_CATEGORY, '^Home - ', '') AS OFFICE_SUB_CATEGORY,
    m.OFFICE_DEPOT_EST_VIEWS,
    m.OFFICE_DEPOT_EST_PURCHASES,
    m.SIMILARITY_SCORE,
    SNOWFLAKE.CORTEX.COMPLETE(
        'mistral-large',
        CONCAT(
        'We are performing product matching across two datasets. The goal is to determine if the products described by their titles are identical, meaning they represent the same real-world item. ',
            'Focus solely on the product titles when deciding whether the products are a match. Do not base the match decision on sub-category. ',
            'After determining whether the titles match, provide unified values for brand and sub-category. ',
            'For the sub-category, adhere to these guidelines: ',
            '- Assign each product to one of the following categories: ',
            '"Adhesives & Fasteners", "Appliances", "Arts & Crafts", "Audio & Video Equipment", ',
            '"Bags, Briefcases & Luggage", "Batteries & Power", "Beverages & Snacks", "Breakroom Supplies", ',
            '"Cleaning Supplies & Chemicals", "Computer Accessories & Peripherals", "Computers & Tablets", ',
            '"Data Storage", "Education Supplies", "Electronics & Networking", "Facilities Maintenance", ',
            '"Foodservice Supplies", "Furniture", "Games, Toys & Entertainment", "Hand & Power Tools", ',
            '"Healthcare & Safety Supplies", "Home Automation & Security", "Janitorial Supplies", ',
            '"Kitchen Supplies", "Networking & WiFi", "Office Decor & Furniture", "Office Machines", ',
            '"Office Supplies", "Paint Supplies", "Paper Products", "Party Supplies & Decor", ',
            '"Phones & Accessories", "Printer Ink & Toner", "Projectors & Office Machines", ',
            '"Retail & Store Supplies", "Safety Equipment", "Shipping Supplies", "Smart Home & Security", ',
            '"Tools & Hardware", "Writing Supplies & Instruments". ',
            '- Use only these categories and consolidate similar sub-categories into one of the categories above. ',
            '- Remove prefixes like "Home - " or "Office - ". Use only the core category name, e.g., "Paint Supplies" instead of "Home - Paint Supplies". ',
            'Analyze the following data and provide a response with four distinct pieces of information: ',
            '1. A boolean indicating whether the products are a match (true/false). ',
            '2. A unified product title that represents both products if they match, or a concise list of the differences between the titles if they do not match, for example "Size, Color, Quantity". ',
            '3. A unified brand based on the brand of both the products. ',
            '4. A unified sub-category based on the given sub-categories of both products, following the guidelines provided. ',
            'The response should be in the following JSON format: ',
            '{"is_match": true/false, "unified_title": "<unified title or differences>", "unified_brand": "<brand or empty>", "unified_sub_category": "<sub category or empty>"} ',
            'Ensure that the output is **valid JSON**, with no extra spaces or newlines before or after the JSON object. ',
            'If the products are a match, provide unified values for title, brand, and sub-category. ',
            'If the products are not a match, only the "unified_title" should describe the differences; the other fields should be unified or left blank. ',
            'Product 1: "', m.STAPLES_TITLE, '" (Brand 1: "', m.STAPLES_BRAND, '", Sub-Category 1: "', STAPLES_SUB_CATEGORY, '"). ',
            'Product 2: "', m.OFFICE_TITLE, '" (Brand 1: "', m.OFFICE_BRAND, '", Sub-Category 1: "', OFFICE_SUB_CATEGORY, '").'
        )
    ) AS MATCH_DECISION
FROM MATCHED_PRODUCTS m;


From there we must parse the structured json outputs from the LLM into separate columns.

In [ ]:
CREATE OR REPLACE TABLE PROCESSED_MATCHES AS
SELECT
    STAPLES_ID,
    STAPLES_TITLE,
    STAPLES_EST_VIEWS,
    STAPLES_EST_PURCHASES,
    OFFICE_ID,
    OFFICE_TITLE,
    OFFICE_DEPOT_EST_VIEWS,
    OFFICE_DEPOT_EST_PURCHASES,
    SIMILARITY_SCORE,
    MATCH_DECISION,
    TRY_PARSE_JSON(
        REGEXP_REPLACE(MATCH_DECISION, '^\n', '')
    ):is_match::BOOLEAN AS IS_MATCH,
    TRY_PARSE_JSON(
        REGEXP_REPLACE(MATCH_DECISION, '^\n', '')
    ):unified_title::STRING AS MATCH_DETAILS,
    TRY_PARSE_JSON(
        REGEXP_REPLACE(MATCH_DECISION, '^\n', '')
    ):unified_brand::STRING AS UNIFIED_BRAND,
    TRY_PARSE_JSON(
        REGEXP_REPLACE(MATCH_DECISION, '^\n', '')
    ):unified_sub_category::STRING AS UNIFIED_CATEGORY
FROM REVIEWED_MATCHES
WHERE TRY_PARSE_JSON(
        REGEXP_REPLACE(MATCH_DECISION, '^\n', '')
    ) IS NOT NULL;


And now, let's take a look at our processed matches:

In [ ]:
SELECT *
FROM PROCESSED_MATCHES
limit 100

Finally, we are splitting our processed matches into tables based on if they were approved or rejected!

The approved product matches list their unified titles, whereas the rejected matches list the differences the LLM flagged between them.

In [ ]:
CREATE OR REPLACE TABLE APPROVED_MATCHES
AS 
SELECT 
STAPLES_ID,
STAPLES_TITLE,
STAPLES_EST_VIEWS,
STAPLES_EST_PURCHASES,
OFFICE_ID,
OFFICE_TITLE,
OFFICE_DEPOT_EST_VIEWS,
OFFICE_DEPOT_EST_PURCHASES,
MATCH_DETAILS AS UNIFIED_TITLE,
UNIFIED_BRAND AS UNIFIED_BRAND,
UNIFIED_CATEGORY AS UNIFIED_CATEGORY
FROM PROCESSED_MATCHES
WHERE IS_MATCH = TRUE

Let's grant access to this table to the analyst role, which is used to run the chatbot in the next section.

In [ ]:
streamlit_grant = f"GRANT INSERT, SELECT, DELETE, UPDATE ON MATCH.APPROVED_MATCHES TO {prefix}_DATA_ANALYST"
session.sql(streamlit_grant).collect()

In [ ]:
SELECT *
from APPROVED_MATCHES
limit 100;

In [ ]:
CREATE OR REPLACE TABLE REJECTED_MATCHES
AS 
SELECT 
STAPLES_ID,
STAPLES_TITLE,
STAPLES_EST_VIEWS,
STAPLES_EST_PURCHASES,
OFFICE_ID,
OFFICE_TITLE,
OFFICE_DEPOT_EST_VIEWS,
OFFICE_DEPOT_EST_PURCHASES,
MATCH_DETAILS AS PRODUCT_DIFFERENCES,
UNIFIED_BRAND AS UNIFIED_BRAND,
UNIFIED_CATEGORY AS UNIFIED_CATEGORY
FROM PROCESSED_MATCHES
WHERE IS_MATCH = FALSE

In [ ]:
SELECT * 
FROM REJECTED_MATCHES
limit 100;